In [1]:
# http://www.wildml.com/2016/08/rnns-in-tensorflow-a-practical-guide-and-undocumented-features/
# http://learningtensorflow.com/index.html
# http://suriyadeepan.github.io/2016-12-31-practical-seq2seq/

import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

In [2]:
# One hot encoding for each char in 'hello'
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

In [3]:
with tf.variable_scope('one_cell') as scope:
    # One cell RNN input_dim (4) -> output_dim (5)
    hidden_size = 5
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    print(cell.output_size, cell.state_size)

    x_data = np.array([[h]], dtype=np.float32) # x_data = [[[1,0,0,0]]]
    pp.pprint(x_data)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

5 5
array([[[ 1.,  0.,  0.,  0.]]], dtype=float32)
array([[[-0.44781727, -0.47987124, -0.26827306,  0.31146127, -0.55739635]]], dtype=float32)


In [4]:
with tf.variable_scope('two_sequances') as scope:
    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5
    hidden_size = 5
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    x_data = np.array([[h, e, l, l, o]], dtype=np.float32)
    pp.pprint(x_data)
    outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]]], dtype=float32)
array([[[  1.49680585e-01,   4.57575798e-01,  -3.26761574e-01,
           6.17544577e-02,  -2.06688106e-01],
        [  9.73679053e-05,   5.36672771e-01,   1.17760032e-01,
          -6.49058297e-02,   5.46318889e-01],
        [  2.76892722e-01,   3.55790615e-01,  -4.51711059e-01,
           2.93671340e-01,   8.11443865e-01],
        [  3.39017272e-01,   4.72602904e-01,  -5.00574112e-01,
           4.61370111e-01,   8.59482050e-01],
        [ -2.57724971e-01,   7.87769735e-01,  -3.28987956e-01,
           3.32676560e-01,   7.03333855e-01]]], dtype=float32)


In [5]:
with tf.variable_scope('3_batches') as scope:
    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    outputs, _states = tf.nn.dynamic_rnn(
        cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)
array([[[-0.07252847,  0.0775635 ,  0.06614664,  0.08079588,  0.04236094],
        [ 0.04452875,  0.06366129,  0.07129606,  0.03916293,  0.10729305],
        [ 0.03708347,  0.06362158,  0.12195665,  0.07282814,  0.01950728],
        [ 0.01745722,  0.07022832,  0.16407327,  0.09803926, -0.02463623],
        [-0.00448285, -0.00953624,  0.12360945,  0.12154452, -0.07512   ]],

       [[ 0.08130765, -0.00799185,  0.03515827, -0.02814181,  0.05611193],
        [ 0.05239937, -0.0723763 ,  0.04340763,  0.01433225, -0.01099364],
  

In [6]:
with tf.variable_scope('3_batches_dynamic_length') as scope:
    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(
        cell, x_data, sequence_length=[5,3,4], dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)
array([[[ 0.0519165 , -0.04327231, -0.08364201, -0.07476711,  0.00534009],
        [ 0.04659839, -0.06371045, -0.19579704, -0.12149881, -0.05449938],
        [ 0.09058722, -0.04613225, -0.06458014, -0.16657713, -0.0457782 ],
        [ 0.1340746 , -0.04038832,  0.0169525 , -0.20342046, -0.02785051],
        [ 0.12421186, -0.0861579 ,  0.00590167, -0.12348987,  0.1156553 ]],

       [[ 0.01386391, -0.03811208, -0.12755112, -0.08774755, -0.05847118],
        [ 0.02469022, -0.08621137, -0.05488021,  0.00699097,  0.0603526 ],
  

In [7]:
with tf.variable_scope('initial_state') as scope:
    batch_size = 3
    x_data = np.array([[h, e, l, l, o],
                      [e, o, l, l, l],
                      [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch: 3
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data,
                                         initial_state=initial_state, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)
array([[[ 0.01048953,  0.07775675, -0.01061853,  0.06709377, -0.09401938],
        [-0.02206461,  0.14650168,  0.06800876,  0.07117976, -0.09127868],
        [-0.11707038,  0.17186792,  0.07723564,  0.07134711, -0.15715575],
        [-0.17756338,  0.19500808,  0.09822351,  0.07682182, -0.19396444],
        [-0.24747425,  0.11276656,  0.08813035,  0.11583821, -0.16482027]],

       [[-0.02317488,  0.08674409,  0.07696244,  0.03443838, -0.03646074],
        [-0.1409598 ,  0.03053352,  0.05721698,  0.08023254, -0.07483447],
  

In [8]:
# Create input data
batch_size=3
sequence_length=5
input_dim=3

x_data = np.arange(45, dtype=np.float32).reshape(batch_size, sequence_length, input_dim)
pp.pprint(x_data)  # batch, sequence_length, input_dim

array([[[  0.,   1.,   2.],
        [  3.,   4.,   5.],
        [  6.,   7.,   8.],
        [  9.,  10.,  11.],
        [ 12.,  13.,  14.]],

       [[ 15.,  16.,  17.],
        [ 18.,  19.,  20.],
        [ 21.,  22.,  23.],
        [ 24.,  25.,  26.],
        [ 27.,  28.,  29.]],

       [[ 30.,  31.,  32.],
        [ 33.,  34.,  35.],
        [ 36.,  37.,  38.],
        [ 39.,  40.,  41.],
        [ 42.,  43.,  44.]]], dtype=float32)


In [9]:
with tf.variable_scope('generated_data') as scope:
    # One cell RNN input_dim (3) -> output_dim (5). sequence: 5, batch: 3
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data,
                                         initial_state=initial_state, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ -6.20050430e-02,   1.94930911e-01,   6.65694550e-02,
          -3.58048603e-02,   4.28726114e-02],
        [ -2.45342590e-02,   2.39187196e-01,  -1.17859460e-01,
          -7.53940176e-03,   7.54191428e-02],
        [ -2.33620685e-03,   1.25779077e-01,  -4.28574443e-01,
          -9.16432356e-04,   7.23467171e-02],
        [ -8.00368507e-05,   3.81559059e-02,  -5.89959681e-01,
          -9.55040523e-05,   5.99204488e-02],
        [ -2.67639825e-06,   7.02464581e-03,  -6.72809839e-01,
          -8.61920944e-06,   4.47936840e-02]],

       [[ -1.49822441e-07,   4.69659833e-04,  -7.13899076e-01,
          -5.71295857e-07,   2.45810430e-02],
        [ -7.07750347e-09,   1.13169284e-04,  -7.38982975e-01,
          -6.15516953e-08,   2.31434256e-02],
        [ -3.92506944e-10,   2.59797253e-05,  -7.50352740e-01,
          -5.10062081e-09,   1.62533764e-02],
        [ -2.17674715e-11,   6.27894769e-06,  -7.56016850e-01,
          -4.18352575e-10,   1.13154007e-02],
        [ -1.2100

In [10]:
with tf.variable_scope('MultiRNNCell') as scope:
    # Make rnn
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    cell = rnn.MultiRNNCell([cell] * 3, state_is_tuple=True) # 3 layers

    # rnn in/out
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    print("dynamic rnn: ", outputs)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())  # batch size, unrolling (time), hidden_size

dynamic rnn:  Tensor("MultiRNNCell/rnn/transpose:0", shape=(3, 5, 5), dtype=float32)
array([[[  9.99064068e-04,  -2.11817445e-03,   1.44370680e-03,
           1.50063180e-03,   1.82173401e-03],
        [  2.46295147e-03,  -4.44928790e-03,   3.61538562e-03,
           3.70276137e-03,   4.01348574e-03],
        [  4.16039489e-03,  -5.28941676e-03,   5.33266086e-03,
           5.84180467e-03,   5.37146395e-03],
        [  5.81860030e-03,  -4.64185700e-03,   6.37572864e-03,
           7.45297596e-03,   5.72169898e-03],
        [  7.21729361e-03,  -3.15448432e-03,   6.95604645e-03,
           8.38801358e-03,   5.32921217e-03]],

       [[  7.49339670e-05,  -8.02440409e-05,   1.47476225e-04,
           3.11960321e-05,   1.99810544e-04],
        [  1.64752928e-04,  -9.17961806e-05,   2.68404925e-04,
           6.32375450e-05,   3.62097402e-04],
        [  2.46637210e-04,  -4.22095836e-05,   3.38729616e-04,
           8.20595887e-05,   4.44179255e-04],
        [  3.11693177e-04,   3.35304685e-

In [11]:
with tf.variable_scope('dynamic_rnn') as scope:
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32,
                                         sequence_length=[1, 3, 2])
    # lentgh 1 for batch 1, lentgh 2 for batch 2
    
    print("dynamic rnn: ", outputs)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())  # batch size, unrolling (time), hidden_size

dynamic rnn:  Tensor("dynamic_rnn/rnn/transpose:0", shape=(3, 5, 5), dtype=float32)
array([[[ -4.44427226e-03,  -1.75914958e-01,   1.76765263e-01,
           1.49680004e-02,   4.00400795e-02],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]],

       [[  5.99127361e-07,  -6.83930889e-03,   4.28351641e-01,
          -2.01454368e-05,   2.11786618e-03],
        [  7.21270794e-08,  -1.01151206e-02,   7.27387071e-01,
          -3.15568582e-06,   7.76955741e-04],
        [  5.94313088e-09,  -1.16600012e-02,   8.82686734e-01,
          -4.60544243e-07,   2.35174608e-04],
        [  0.00000000e+00,   0.00000000e+0

In [12]:
with tf.variable_scope('bi-directional') as scope:
    # bi-directional rnn
    cell_fw = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    cell_bw = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)

    outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, x_data,
                                                      sequence_length=[2, 3, 1],
                                                      dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    pp.pprint(sess.run(outputs))
    pp.pprint(sess.run(states))

(   array([[[ -1.97037429e-01,   3.19136083e-01,   1.21739082e-01,
           7.28317574e-02,   5.46695525e-03],
        [ -4.64550942e-01,   2.38791093e-01,   2.29666293e-01,
           2.73308009e-01,   3.25907432e-02],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]],

       [[ -7.15138733e-01,   1.78781373e-03,   3.21433872e-01,
           2.11304612e-02,   1.54711415e-05],
        [ -7.27850437e-01,   6.33687479e-04,   3.50840300e-01,
           1.33414185e-02,   1.82280110e-06],
        [ -7.38869429e-01,   1.81801879e-04,   3.31723392e-01,
           6.80072745e-03,   2.47312755e-07],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.

In [13]:
# flattern based softmax
hidden_size=3
sequence_length=5
batch_size=3
num_classes=5

pp.pprint(x_data) # hidden_size=3, sequence_length=4, batch_size=2
x_data = x_data.reshape(-1, hidden_size)
pp.pprint(x_data)

softmax_w = np.arange(15, dtype=np.float32).reshape(hidden_size, num_classes)
outputs = np.matmul(x_data, softmax_w)
outputs = outputs.reshape(-1, sequence_length, num_classes) # batch, seq, class
pp.pprint(outputs)

array([[[  0.,   1.,   2.],
        [  3.,   4.,   5.],
        [  6.,   7.,   8.],
        [  9.,  10.,  11.],
        [ 12.,  13.,  14.]],

       [[ 15.,  16.,  17.],
        [ 18.,  19.,  20.],
        [ 21.,  22.,  23.],
        [ 24.,  25.,  26.],
        [ 27.,  28.,  29.]],

       [[ 30.,  31.,  32.],
        [ 33.,  34.,  35.],
        [ 36.,  37.,  38.],
        [ 39.,  40.,  41.],
        [ 42.,  43.,  44.]]], dtype=float32)
array([[  0.,   1.,   2.],
       [  3.,   4.,   5.],
       [  6.,   7.,   8.],
       [  9.,  10.,  11.],
       [ 12.,  13.,  14.],
       [ 15.,  16.,  17.],
       [ 18.,  19.,  20.],
       [ 21.,  22.,  23.],
       [ 24.,  25.,  26.],
       [ 27.,  28.,  29.],
       [ 30.,  31.,  32.],
       [ 33.,  34.,  35.],
       [ 36.,  37.,  38.],
       [ 39.,  40.,  41.],
       [ 42.,  43.,  44.]], dtype=float32)
array([[[   25.,    28.,    31.,    34.,    37.],
        [   70.,    82.,    94.,   106.,   118.],
        [  115.,   136.,   157.,   178

In [14]:
# [batch_size, sequence_length, emb_dim ]
prediction1 = tf.constant([[[0, 1], [0, 1], [0, 1]]], dtype=tf.float32)
prediction2 = tf.constant([[[1, 0], [1, 0], [1, 0]]], dtype=tf.float32)
prediction3 = tf.constant([[[0, 1], [1, 0], [0, 1]]], dtype=tf.float32)

# [batch_size, sequence_length]
y_data = tf.constant([[1, 1, 1]])

# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

sequence_loss1 = tf.contrib.seq2seq.sequence_loss(prediction1, y_data, weights)
sequence_loss2 = tf.contrib.seq2seq.sequence_loss(prediction2, y_data, weights)
sequence_loss3 = tf.contrib.seq2seq.sequence_loss(prediction3, y_data, weights)

sess.run(tf.global_variables_initializer())
print("Loss1: ", sequence_loss1.eval(),
      "Loss2: ", sequence_loss2.eval(),
      "Loss3: ", sequence_loss3.eval())

Loss1:  0.313262 Loss2:  1.31326 Loss3:  0.646595
